In [ ]:
import pandas as pd

path = 'Originals/'
path_to_save = 'Data curated/'

file = 'A_ProblemasPacientes.txt'
df_diagnostics = pd.read_csv(path + file, encoding='ansi', sep='|' )

# Drop records

## Duplicated

In [ ]:
dup = df_diagnostics.duplicated()
df_diagnostics = df_diagnostics[~dup]
df_diagnostics.reset_index(inplace=True, drop=True)

## Records with 'Code' as NaN

In [ ]:
df_diagnostics = df_diagnostics[df_diagnostics['Code'].notnull()]
df_diagnostics.reset_index(inplace=True, drop=True)

# Translation of values

## Column names

In [ ]:
column_name_english = ['Patient_ID', 'Code', 'Description']
df_diagnostics.columns = column_name_english

## Translate from CIE-9MC to ICD9

### Sort dataset by 'Code'

In [ ]:
df_diagnostics.sort_values(by=['Code'], inplace=True)
df_diagnostics.reset_index(inplace=True, drop=True)

### Extracting codes from standard ICD-9

In [ ]:
df_codes = pd.read_excel('CMS32_DESC_LONG_SHORT_DX.xlsx')

### Sort the code dataset by code too

In [ ]:
df_codes = df_codes.sort_values(by=['DIAGNOSIS CODE'])
df_codes.reset_index(inplace=True, drop=True)

### Translation

In [ ]:
df_diagnostics_converted = df_diagnostics.copy() # Make a copy of the original dataset

for i in range(df_diagnostics_converted.shape[0]): # From i = 0 to the number o rows od the dataset
    current_MY_code = df_diagnostics_converted.iloc[i]['Code'] # Current code from original
    code_ICD9_list = df_codes.index[(df_codes['DIAGNOSIS CODE'] == current_MY_code)].tolist() # List of index that match with this code
    if len(code_ICD9_list) == 1: # If there is a row that match with that code
        description_from_ICD9 = df_codes.iloc[code_ICD9_list[0]]['LONG DESCRIPTION'] # The description from the excel that match with the code.
    else: # The code it is not found in the Excel file
        description_from_ICD9 = 'VALUE NOT FOUND'

    df_diagnostics_converted.loc[i, 'NEW_DESCRIPTION'] = description_from_ICD9 # The new value is assign to the corresponding row, whatever be assign above.

# Sort the dataset by 'Patient_ID' and 'Code'

In [ ]:
df_diagnostics_converted = df_diagnostics_converted.sort_values(by=[ 'Patient_ID', 'Code'])
df_diagnostics_converted.reset_index(inplace=True, drop=True)

# Save data as CSV

In [ ]:
df_diagnostics_converted.to_csv(path_to_save + 'Diagnostics.csv',index=False)